In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from torchvision import transforms
from PIL import Image
import os

In [7]:
training_dataset_dir = "/content/drive/MyDrive/richhf/train" if os.path.exists("/content/drive/MyDrive/richhf/train") else "./train"
train_data_csv = "/content/drive/MyDrive/richhf/RichHF_18K.csv" if os.path.exists("/content/drive/MyDrive/richhf") else "./richhf/RichHF_18K.csv"
model_output_name =  "/content/drive/MyDrive/meta_iqa_model.pth" if os.path.exists("/content/drive/MyDrive") else "./meta_iqa_model.pth"


In [ ]:
class MetaIQAFeatureExtractor(nn.Module):
    def __init__(self, input_size=3*512*512, hidden_size=1024, output_size=512):
        super(MetaIQAFeatureExtractor, self).__init__()
        self.model = nn.Sequential(
        nn.Linear(input_size, hidden_size),
        nn.LeakyReLU(),  
        nn.Dropout(0.3),
        nn.Linear(hidden_size, output_size),
        nn.LeakyReLU(),
        nn.Linear(output_size, 1)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  
        return self.model(x)




In [20]:
class IQADataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        # Get image name and quality score from the dataframe
        img_name = self.data_frame.iloc[idx, 0]
        quality_score = self.data_frame.iloc[idx, 5]

        img_path = f"{self.root_dir}/{img_name}.png"

        # Check if the image exists only when loading it
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Image {img_path} not found")

        # Load the image lazily
        image = Image.open(img_path).convert('RGB')

        # Apply transform if necessary
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(quality_score, dtype=torch.float)


In [21]:
def train_meta_iqa(model, train_loader, optimizer, criterion, device, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, quality_scores in train_loader:
            images, quality_scores = images.to(device), quality_scores.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, quality_scores.unsqueeze(1))  # Ensure dimensions match
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")


In [22]:
if __name__ == "__main__":
    # Hyperparameters and configuration
    input_size = 3*512*512
    hidden_size = 1024
    output_size = 512
    epochs = 10
    batch_size = 32
    learning_rate = 1e-4

    # Prepare the dataset and dataloader
    transform = transforms.Compose([transforms.Resize((512, 512)), transforms.ToTensor()])

    dataset = IQADataset(csv_file=train_data_csv, root_dir=training_dataset_dir, transform=transform)
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Initialize the model, optimizer, and loss function
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MetaIQAFeatureExtractor(input_size=input_size, hidden_size=hidden_size, output_size=output_size).to(device)
    optimizer = Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()  # Mean Squared Error Loss

    # Train the model
    train_meta_iqa(model, train_loader, optimizer, criterion, device, epochs)

    # Save the trained model
    
    torch.save(model.state_dict(), model_output_name)
    print("Training complete and model saved!")

Epoch [1/10], Loss: 4.5805
Epoch [2/10], Loss: 0.0388
Epoch [3/10], Loss: 0.0335
Epoch [4/10], Loss: 0.0325
Epoch [5/10], Loss: 0.0402
Epoch [6/10], Loss: 0.0304
Epoch [7/10], Loss: 0.0308
Epoch [8/10], Loss: 0.0314
Epoch [9/10], Loss: 0.6506
Epoch [10/10], Loss: 0.1571
Training complete and model saved!
